In [123]:
from dotenv import dotenv_values
config = dotenv_values(".env")

In [124]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(dbname=config['DBNAME'], user=config['USER'], password=config['PASSWORD'])

In [125]:
# Get last date tables
df_tables = pd.read_sql('SELECT table_name FROM information_schema.tables WHERE table_schema=\'' + config['SCHEMA'] +'\'   AND table_type=\'BASE TABLE\';', conn)
df_tables

,table_name
0,morticd_10_1_f695_1642950107
1,morticd_10_3_1c6a_1642928618
2,morticd_10_4_453e_1642928618
3,notes_0_a67b_1642928619
4,population_0_3b8f_1642928619
5,demographiccountry_0_e0e0_1642950106
6,demographic_0_76f7_1642950106
7,demographiclabels_0_e8a5_1642950107
8,morticd_7_22b4_1642950107
9,morticd_8_9366_1642950107


In [126]:
max_timestamp = df_tables['table_name'].apply(lambda x: x[x.rfind('_') +1 :-2]).max()
max_timestamp


'16429501'

In [127]:
df_lastupdate_tables = df_tables['table_name'].apply(lambda x: max_timestamp in x)
df_tables[df_lastupdate_tables]

,table_name
0,morticd_10_1_f695_1642950107
5,demographiccountry_0_e0e0_1642950106
6,demographic_0_76f7_1642950106
7,demographiclabels_0_e8a5_1642950107
8,morticd_7_22b4_1642950107
9,morticd_8_9366_1642950107
10,morticd_9_71c9_1642950107
11,morticd_10_2_ffb1_1642950108
12,morticd_10_3_1c6a_1642950108
13,morticd_10_4_453e_1642950109


In [128]:
mortic_dataframes = []
demo_dataframes = []
general_dataframes = []
for table in df_tables[df_lastupdate_tables]['table_name']:
    firstDash = str(table).index('_', 0)
    lastDash = str(table).rindex('_')
    secondlastDash = str(table).rindex('_', lastDash)
    dfname = str(table[:secondlastDash])
    if  dfname.count('_') >= 2:
        lastDash = dfname.rindex('_')
        dfname = dfname[:lastDash]
    if 'morticd' in dfname:
        mortic_dataframes.append('df_' + dfname)
    elif 'demo' in dfname:
        demo_dataframes.append('df_' + dfname)
    else:
        general_dataframes.append('df_' + dfname)
    globals()['df_' + dfname] = pd.read_sql('SELECT * from ' + config['SCHEMA'] + '.' + table + ';', conn)

In [129]:
demo_dataframes

['df_demographiccountry_0', 'df_demographic_0', 'df_demographiclabels_0']

In [130]:
mortic_dataframes

['df_morticd_10_1',
 'df_morticd_7',
 'df_morticd_8',
 'df_morticd_9',
 'df_morticd_10_2',
 'df_morticd_10_3',
 'df_morticd_10_4']

In [131]:
general_dataframes

['df_countrycodes_0', 'df_notes_0', 'df_population_0']

In [132]:
# Check if morticd countries are in countrycode country
def getCountryCodedf():
    for name in globals().keys():
        if 'countrycode' in name: 
            return name

df_countryCode = getCountryCodedf()
  
for dataframe in mortic_dataframes[0:2]:
    globals()[str(dataframe)]['CountryWrongCode'] = globals()[str(dataframe)]['Country'].apply(lambda x: x in globals()[str(df_countryCode)].Country)

In [133]:
# Check morticd Size
for mortic_dataframe in mortic_dataframes:
    print(mortic_dataframe)
    globals()[str(mortic_dataframe)]['CountryWrongSize'] = globals()[str(mortic_dataframe)]['Country'].apply(lambda x: x != None and len(str(x)) != 4)
    globals()[str(mortic_dataframe)]['Admin1WrongSize'] = globals()[str(mortic_dataframe)]['Admin1'].apply(lambda x: x != None and len(str(x)) != 3)
    globals()[str(mortic_dataframe)]['SubDivWrongSize'] = globals()[str(mortic_dataframe)]['SubDiv'].apply(lambda x: x != None and len(str(x)) != 3)
    globals()[str(mortic_dataframe)]['YearWrongSize'] = globals()[str(mortic_dataframe)]['Year'].apply(lambda x: x != None and len(str(x)) != 4)
    globals()[str(mortic_dataframe)]['ListWrongSize'] = globals()[str(mortic_dataframe)]['List'].apply(lambda x: x != None and len(str(x)) != 3)
    globals()[str(mortic_dataframe)]['CauseWrongSize'] = globals()[str(mortic_dataframe)]['Cause'].apply(lambda x: x != None and len(str(x)) > 4)
    globals()[str(mortic_dataframe)]['SexWrongSize'] = globals()[str(mortic_dataframe)]['Sex'].apply(lambda x: x != None and len(str(x)) != 1)
    globals()[str(mortic_dataframe)]['FrmatWrongSize'] = globals()[str(mortic_dataframe)]['Frmat'].apply(lambda x: x != None and len(str(x)) != 2)
    globals()[str(mortic_dataframe)]['IM_FrmatWrongSize'] = globals()[str(mortic_dataframe)]['IM_Frmat'].apply(lambda x: x != None and len(str(x)) != 2)

df_morticd_10_1


In [134]:
for dataframe in mortic_dataframes[0:1]:
    death_colnames = globals()[str(dataframe)].filter(regex=("Deaths*")).columns

death_colnames

for death in death_colnames:    
    globals()[str(dataframe)][death + 'WrongSize'] = globals()[str(dataframe)][death].apply(lambda x: len(str(x)) > 8)
    globals()[str(dataframe)][death + 'WrongValue'] = globals()[str(dataframe)][death].apply(lambda x: x < 0)   

In [135]:
df_demo_country = pd.read_sql('SELECT * from formatted.demographiccountry_0_e0e0_1642950106;', conn)
df_demo_labels = pd.read_sql('SELECT * from formatted.demographiclabels_0_e8a5_1642950107;', conn)
df_demo_indic = pd.read_sql('SELECT * from formatted.demographic_0_76f7_1642950106;', conn)

In [136]:
# Check if demographic country codes are in demographic country table
for dataframe in demo_dataframes:
    globals()[str(dataframe)]['CountryWrongId'] = globals()[str(dataframe)]['CountryId'].apply(lambda x: x in df_demo_country.Code)

In [137]:
# Check if demographic Indicator ID are in demographic label table
for dataframe in demo_dataframes:
    globals()[str(dataframe)]['IndicatorIdWrongId'] = globals()[str(dataframe)]['IndicatorId'].apply(lambda x: x in df_demo_labels.Indicator)

In [138]:
# Check size Demographic label columns
for dataframe in demo_dataframes:
    globals()[str(dataframe)]['IndicatorWrongSize'] = globals()[str(dataframe)]['Indicator'].apply(lambda x: x != None and len(str(x))  != 50)
    globals()[str(dataframe)]['DescriptionWrongSize'] = globals()[str(dataframe)]['Description'].apply(lambda x: x != None and len(str(x))  != 100)

In [139]:
# Check size Demographic country columns
for dataframe in demo_dataframes:
    globals()[str(dataframe)]['CodeWrongSize'] = globals()[str(dataframe)]['Code'].apply(lambda x: x != None and len(str(x))  != 3)
    globals()[str(dataframe)]['NameWrongSize'] = globals()[str(dataframe)]['Name'].apply(lambda x: x != None and len(str(x))  != 100)

In [107]:
# Check size Demographic country columns
for dataframe in demo_dataframes:
    globals()[str(dataframe)]['IndicatorIdWrongSize'] = globals()[str(dataframe)]['IndicatorId'].apply(lambda x: x != None and len(str(x))  != 50)
    globals()[str(dataframe)]['CountryIdWrongSize'] = globals()[str(dataframe)]['CountryId'].apply(lambda x: x != None and len(str(x))  != 3)
    globals()[str(dataframe)]['YearWrongSize'] = globals()[str(dataframe)]['Year'].apply(lambda x: x != None and len(str(x))  != 4)
    globals()[str(dataframe)]['MagnitudeWrongSize'] = globals()[str(dataframe)]['Magnitude'].apply(lambda x: x != None and len(str(x))  != 50)
    globals()[str(dataframe)]['QualifierWrongSize'] = globals()[str(dataframe)]['Qualifier'].apply(lambda x: x != None and len(str(x))  != 50)

In [143]:
# Check Uniques Demographic Country
def getDemoCountrydf():
    for name in globals().keys():
        if 'demographiccountry' in name: 
            return name

df_demo_country = getDemoCountrydf()

df_demo_country_unique = pd.DataFrame(globals()[str(df_demo_country)]['Code'].value_counts())
df_demo_country['CodeWrongUnique'] = df_demo_country_unique['Code'] > 2

df_demographiccountry_0


In [147]:
# Check Uniques Demographic Labels
def getDemoLabelsdf():
    for name in globals().keys():
        if 'demographiclabels' in name: 
            return name

df_demo_labels_code = getDemoLabelsdf()

df_demo_labels_unique = pd.DataFrame(globals()[str(df_demo_labels_code)]['Indicator'].value_counts())
df_demo_labels['IndicatorWrongUnique'] = df_demo_labels_unique['Indicator'] > 2

In [148]:
# Check Uniques Country codes
df_country_code_unique = pd.DataFrame(globals()[str(df_countryCode)]['Country'].value_counts())
df_country_code_unique['CountryWrongUnique'] = df_country_code_unique['Country'] > 2

In [152]:
for dataframe in mortic_dataframes[0:1]:
    wrongs_mortic = globals()[str(dataframe)].filter(regex=(".Wrong*")).columns
wrongs_mortic

Index(['CountryWrongCode', 'CountryWrongSize', 'Admin1WrongSize',
       'SubDivWrongSize', 'YearWrongSize', 'ListWrongSize', 'CauseWrongSize',
       'SexWrongSize', 'FrmatWrongSize', 'IM_FrmatWrongSize',
       'Deaths1WrongSize', 'Deaths1WrongValue', 'Deaths2WrongSize',
       'Deaths2WrongValue', 'Deaths3WrongSize', 'Deaths3WrongValue',
       'Deaths4WrongSize', 'Deaths4WrongValue', 'Deaths5WrongSize',
       'Deaths5WrongValue', 'Deaths6WrongSize', 'Deaths6WrongValue',
       'Deaths7WrongSize', 'Deaths7WrongValue', 'Deaths8WrongSize',
       'Deaths8WrongValue', 'Deaths9WrongSize', 'Deaths9WrongValue',
       'Deaths10WrongSize', 'Deaths10WrongValue', 'Deaths11WrongSize',
       'Deaths11WrongValue', 'Deaths12WrongSize', 'Deaths12WrongValue',
       'Deaths13WrongSize', 'Deaths13WrongValue', 'Deaths14WrongSize',
       'Deaths14WrongValue', 'Deaths15WrongSize', 'Deaths15WrongValue',
       'Deaths16WrongSize', 'Deaths16WrongValue', 'Deaths17WrongSize',
       'Deaths17WrongValue

In [153]:
for dataframe in demo_dataframes[0:1]:
    wrongs_demo = globals()[str(dataframe)].filter(regex=(".Wrong*")).columns
wrongs_demo

Index([], dtype='object')

In [154]:
for dataframe in mortic_dataframes[0:1]:
   for wrong_col in wrongs_mortic:
      if len(globals()[str(dataframe)][globals()[str(dataframe)][wrong_col] == True]) > 0:
         print(wrong_col)

In [155]:
for dataframe in demo_dataframes[0:1]:
   for wrong_col in wrongs_demo:
      if len(globals()[str(dataframe)][globals()[str(dataframe)][wrong_col] == True]) > 0:
         print(wrong_col)